In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
def load_result(file_path):
  results = []
  with open(file_path, "r") as f:
    lines = f.readlines()
    for line in lines:
      result_str = line.split(",")[:-1]
      result_int = []
      for tag in result_str:
        result_int.append(int(tag))
      results.append(result_int)
  return results

def load_result_2(file_path):
  results = []
  with open(file_path, "r") as f:
    lines = f.readlines()
    for line in lines:
      result_str = line.replace("\"", "").replace("[", "").replace("]", "").replace("\n", "").split(",")
      result_int = []
      for tag in result_str:
        if tag != "":
          result_int.append(int(tag))
      results.append(result_int)
  return results

In [ ]:
bert = load_result("/content/drive/My Drive/5018_ToxicSpans/label_files/bert.csv")
bert_avg = load_result("/content/drive/My Drive/5018_ToxicSpans/label_files/bert_avg.csv")
bert_lstm = load_result("/content/drive/My Drive/5018_ToxicSpans/label_files/bert_lstm.csv")
bert_pseudo = load_result("/content/drive/My Drive/5018_ToxicSpans/label_files/bert_pseudo.csv")
flair = load_result_2("/content/drive/My Drive/5018_ToxicSpans/label_files/flair.txt")
fasttext = load_result_2("/content/drive/My Drive/5018_ToxicSpans/label_files/fasttext.txt")

# # Note:
# # flair.txt is renamed from /pred_result/flair_forweard_pred.csv and the first line is removed.
# # fasttext.txt is renamed from /pred_result/ft_pred.csv and the first line is removed.

In [ ]:
#@title fix spans { form-width: "1px" }
import itertools
import string 
import csv
import ast

SPECIAL_CHARACTERS = string.whitespace

def _contiguous_ranges(span_list):
    """Extracts continguous runs [1, 2, 3, 5, 6, 7] -> [(1,3), (5,7)]."""
    output = []
    for _, span in itertools.groupby(
        enumerate(span_list), lambda p: p[1] - p[0]):
        span = list(span)
        output.append((span[0][1], span[-1][1]))
    return output

def fix_spans(spans, text, special_characters=SPECIAL_CHARACTERS):
    """Applies minor edits to trim spans and remove singletons."""
    cleaned = []
    for begin, end in _contiguous_ranges(spans):
        while text[begin] in special_characters and begin < end:
            begin += 1
        while text[end] in special_characters and begin < end:
            end -= 1
        if end - begin > 1:
            cleaned.extend(range(begin, end + 1))
    return cleaned

def read_datafile(filename):
  """Reads csv file with python span list and text."""
  data = []
  with open(filename) as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
      fixed = fix_spans(
          ast.literal_eval(row['spans']), row['text'])
      data.append((fixed, row['text']))
  return data

test= read_datafile('/content/drive/My Drive/5018_ToxicSpans/datasets/tsd_test.csv')
true_labels = []
for i in range(len(test)):
  true_labels.append(test[i][0])

In [ ]:
import numpy as np

def f1(predictions, gold):
  if len(gold) == 0:
    return 1. if len(predictions) == 0 else 0.
  if len(predictions) == 0:
    return 0.
  predictions_set = set(predictions)
  gold_set = set(gold)
  nom = 2 * len(predictions_set.intersection(gold_set))
  denom = len(predictions_set) + len(gold_set)
  return float(nom)/float(denom)

pred_labels = []
for i in range(len(bert)):
  labels = []
  tmp = []
  # tmp.extend(bert[i])
  # tmp.extend(bert_pseudo[i])
  # tmp.extend(bert_avg[i])
  # tmp.extend(bert_lstm[i])
  # tmp.extend(fasttext[i])
  tmp.extend(flair[i])
  
  set_tag = set(tmp)

  # # vote (>50%)
  # for tag in set_tag:
  #   if tmp.count(tag) > 1:
  #     labels.append(tag)

  # # union
  # labels = list(set_tag)

  # intersection
  for tag in set_tag:
    if tmp.count(tag) > 0:
      labels.append(tag)

  pred_labels.append(labels)

scores = [] 
for i in range(len(true_labels)):
  scores.append(f1(pred_labels[i], true_labels[i]))
scores = np.mean(scores)
print("test f1: {}".format(scores * 100))

test f1: 65.50818381917338


# # best 68.43:
# intersection(bert_avg, bert_lstm, fasttext, flair)